# PETScanv1.0

hier finden sie eine Pipeline zur sequenziellen verarbeitung von Bilddaten zur Detektion mangelhafter Exemplare im Hochgeschwindigkeitsfluss

Daten vorbereiten (Train/Test-Split)

🔹 Ziel: Die vorhandenen 50 i.O.- und 40 n.i.O.-Bilder aufteilen in Trainings- und Testdaten.
🔹 Vorgehen:

    80% der Bilder zum Trainieren des Modells
    20% der Bilder für den Test (damit wir das Modell später evaluieren können)

In [1]:
import os
import shutil
import random

# Ordner mit den Originalbildern
IO_PATH = "C:/Users/ThinkPad/Bewerbung/abgeschickt/Pro Projekte/Bilder/i.O"
NIO_PATH = "C:/Users/ThinkPad/Bewerbung/abgeschickt/Pro Projekte/Bilder/n.i.O"

# Zielordner für Training & Test
TRAIN_PATH = "C:/Users/ThinkPad/Bewerbung/abgeschickt/Pro Projekte/Bilder/train"
TEST_PATH = "C:/Users/ThinkPad/Bewerbung/abgeschickt/Pro Projekte/Bilder/test"

# Prozentsatz für Training (z. B. 80% Training, 20% Test)
TRAIN_SPLIT = 0.8

# Sicherstellen, dass Ordner existieren
for path in [TRAIN_PATH, TEST_PATH]:
    for category in ["iO", "niO"]:
        os.makedirs(os.path.join(path, category), exist_ok=True)

# Funktion zum Aufteilen der Bilder
def split_images(source_folder, train_folder, test_folder, category):
    images = os.listdir(source_folder)
    random.shuffle(images)  # Zufällige Reihenfolge

    train_count = int(len(images) * TRAIN_SPLIT)
    train_images = images[:train_count]
    test_images = images[train_count:]

    # Bilder in die entsprechenden Ordner kopieren
    for img in train_images:
        shutil.copy(os.path.join(source_folder, img), os.path.join(train_folder, category, img))
    for img in test_images:
        shutil.copy(os.path.join(source_folder, img), os.path.join(test_folder, category, img))

# Bilder aufteilen
split_images(IO_PATH, TRAIN_PATH, TEST_PATH, "iO")
split_images(NIO_PATH, TRAIN_PATH, TEST_PATH, "niO")

print("✅ Bilder wurden erfolgreich aufgeteilt!")


✅ Bilder wurden erfolgreich aufgeteilt!


Ziel der Bildvorverarbeitung

Wir wollen sicherstellen, dass unsere Bilder einheitlich und optimal für das Modell aufbereitet sind.
Das bedeutet:

✅ Flasche im Bild erkennen & ausschneiden (ROI-Extraktion)
✅ Bilder auf eine feste Größe bringen (224x224 Pixel für CNNs)
✅ Farbwerte normalisieren (ohne Graustufen!)
✅ Optional: Augmentation für mehr Datenvielfalt

In [2]:
import cv2
import os
import numpy as np
import matplotlib.pyplot as plt

# Pfade zu den Trainingsbildern
TRAIN_PATH = "C:/Users/ThinkPad/Bewerbung/abgeschickt/Pro Projekte/Bilder/train"
PROCESSED_PATH = "C:/Users/ThinkPad/Bewerbung/abgeschickt/Pro Projekte/Bilder/processed"

# Sicherstellen, dass der Verarbeitungsordner existiert
for category in ["iO", "niO"]:
    os.makedirs(os.path.join(PROCESSED_PATH, category), exist_ok=True)

# Funktion zur ROI-Extraktion (Flasche finden und ausschneiden)
def extract_bottle_roi(image):
    # Bild in HSV umwandeln (für bessere Farberkennung)
    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)

    # Grüne Flasche erkennen (HSV-Farbfilter)
    lower_green = np.array([35, 40, 40])  # Untere Grenze für Grün
    upper_green = np.array([85, 255, 255])  # Obere Grenze für Grün
    mask = cv2.inRange(hsv, lower_green, upper_green)

    # Konturen finden
    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Falls Konturen gefunden wurden, wähle die größte
    if contours:
        largest_contour = max(contours, key=cv2.contourArea)
        x, y, w, h = cv2.boundingRect(largest_contour)
        roi = image[y:y+h, x:x+w]  # Region of Interest (Flasche)
        return roi
    else:
        return None  # Keine Flasche gefunden

# Verarbeite alle Bilder im Trainingsset
for category in ["iO", "niO"]:
    folder_path = os.path.join(TRAIN_PATH, category)
    processed_folder = os.path.join(PROCESSED_PATH, category)

    for filename in os.listdir(folder_path):
        img_path = os.path.join(folder_path, filename)
        image = cv2.imread(img_path)

        if image is not None:
            roi = extract_bottle_roi(image)  # Flasche extrahieren
            if roi is not None:
                roi_resized = cv2.resize(roi, (224, 224))  # Bildgröße anpassen
                cv2.imwrite(os.path.join(processed_folder, filename), roi_resized)  # Speichern
            else:
                print(f"⚠ Keine Flasche erkannt in: {filename}")


ModuleNotFoundError: No module named 'cv2'